In [1]:
import os
import shutil
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.preprocessing import image as image_prepocessor
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential  
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D 
from keras.regularizers import l2

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_path = "/home/paperspace/data/dogBreed/"
# data_path = r"E:\DogBreed"
im_size = 225
batch_size = 64
epochs = 50
_seed = 1

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
model = ResNet50(include_top=False, weights='imagenet')

In [4]:
_training_image_loc = os.path.join(data_path, "train")
_training_label_loc = os.path.join(data_path, "labels.csv")
_testing_image_loc = os.path.join(data_path, "test")
_sample_sub_loc = os.path.join(data_path, "sample_submission.csv")

In [5]:
temp_dir = "temp"
if not os.path.exists(temp_dir):
    os.mkdir(temp_dir)
    
bottle_ft_train_path = os.path.join(temp_dir,'btn_ft_train_{}_{}.npy'.format(im_size, batch_size))
bottle_ft_val_path = os.path.join(temp_dir,'btn_ft_val_{}_{}.npy'.format(im_size, batch_size))

In [6]:
def split_images_into_breed(dataframe, new_dir_name):
    sub_path = os.path.join(_training_image_loc, new_dir_name)
    
    if os.path.exists(sub_path):
        print("Folder {} exists.".format(sub_path))
        return sub_path
    
    os.mkdir(sub_path)
        
    for name, group in tqdm(dataframe.groupby("breed")):
        breed_dir = os.path.join(sub_path, name)
        if not os.path.exists(breed_dir):
            os.mkdir(breed_dir)
        for file_name in group["id"]:
            file_path = os.path.join(_training_image_loc, file_name + ".jpg")
            if os.path.exists(file_path):
                shutil.copy(file_path, breed_dir)
    return sub_path

In [7]:
df = pd.read_csv(_training_label_loc)

np.random.seed(seed=_seed)
rnd = np.random.random(df.shape[0])
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8

traing_df = df[train_idx]
validation_df = df[valid_idx]
train_data_loc = split_images_into_breed(traing_df, "train")
val_data_loc = split_images_into_breed(validation_df, "valid")

Folder /home/paperspace/data/dogBreed/train/train exists.
Folder /home/paperspace/data/dogBreed/train/valid exists.


In [8]:
data_generator = ImageDataGenerator(rescale=None)
if not os.path.exists(bottle_ft_train_path):
    train_generator = data_generator.flow_from_directory(train_data_loc,
                                                         target_size=(im_size, im_size), 
                                                         batch_size=batch_size,
                                                         class_mode=None,
                                                         shuffle=False)
    num_of_training_samples = len(train_generator.filenames)
    num_classes = len(train_generator.class_indices)
    steps = int(math.ceil(num_of_training_samples / batch_size))  

    bottleneck_features_train = model.predict_generator(train_generator, steps)
    np.save(bottle_ft_train_path, bottleneck_features_train)
else:
    print("Skipping saving of train data.")

Found 8221 images belonging to 120 classes.


In [9]:
if not os.path.exists(bottle_ft_val_path):
    val_generator = data_generator.flow_from_directory(val_data_loc,
                                                       target_size=(im_size, im_size), 
                                                       batch_size=batch_size,
                                                       class_mode=None,
                                                       shuffle=False)
    num_of_val_samples = len(val_generator.filenames)
    steps = int(math.ceil(num_of_val_samples / batch_size))  

    bottleneck_features_val = model.predict_generator(val_generator)
    np.save(bottle_ft_val_path, bottleneck_features_val)  
else:
    print("Skipping saving of validation data.")

Found 2001 images belonging to 120 classes.


In [ ]:
# Generator with images and Labels
train_generator = data_generator.flow_from_directory(train_data_loc,
                                                     target_size=(im_size, im_size), 
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     shuffle=False)
num_of_training_samples = len(train_generator.filenames)
num_classes = len(train_generator.class_indices)

train_data = np.load(bottle_ft_train_path)
train_labels = train_generator.classes.reshape(-1, 1)
encoder = OneHotEncoder()
train_labels = encoder.fit_transform(train_labels)

In [ ]:
val_generator = data_generator.flow_from_directory(val_data_loc,
                                                   target_size=(im_size, im_size), 
                                                   batch_size=batch_size,
                                                   class_mode=None,
                                                   shuffle=False)
num_of_val_samples = len(val_generator.filenames)
validation_data = np.load(bottle_ft_val_path)  
validation_labels = val_generator.classes.reshape(-1, 1)
encoder = OneHotEncoder()
validation_labels = encoder.fit_transform(validation_labels)

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(2048, activation="relu"))
model.add(Dropout(0.8))
# model.add(Dense(256, activation="relu"))
# model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size,
                   validation_data=(validation_data, validation_labels))


In [ ]:
plt.figure(1)  
   
# summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  

# summarize history for loss  

plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()  